# Lab 1.7.2: Layer Implementation

**Module:** 1.7 - Domain 1 Capstone: MicroGrad+  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand how neural network layers are structured
- [ ] Implement Linear, ReLU, Sigmoid, Softmax layers
- [ ] Build a Dropout layer that behaves differently in train/eval mode
- [ ] Combine layers into a Sequential container

---

## 📚 Prerequisites

- Completed: Lab 1.7.1 (Core Tensor Implementation)
- Knowledge of: Neural network basics, activation functions

---

## 🌍 Real-World Context

Layers are the building blocks of neural networks. Understanding how they work helps you:
- Design custom architectures for your specific problems
- Debug training issues ("Why are my gradients zero?")
- Implement novel layers from research papers
- Optimize inference by understanding what each layer does

---

## 🧒 ELI5: Neural Network Layers

> **Imagine a factory assembly line** where each station does one specific job.
>
> - **Linear Layer**: A station that weighs and combines ingredients. Each output is a weighted sum of inputs, like mixing paint colors.
>
> - **ReLU Activation**: A quality control station that only lets positive values through. Negative? Becomes zero. Positive? Passes unchanged.
>
> - **Softmax**: A station that converts scores into percentages. "You scored 80, 90, 70" becomes "You have 24%, 46%, 30% chance of winning."
>
> - **Dropout**: A random vacation scheduler! During training, some workers randomly take the day off, forcing everyone else to learn multiple jobs.
>
> **In AI terms:** Each layer transforms the data in a specific way. Linear layers learn patterns, activations add non-linearity (necessary for complex functions), and regularization layers like Dropout prevent overfitting.

> **Learning Note:**
>
> In this notebook, we implement neural network layers from scratch for educational purposes.
> The complete, tested implementations are already available in the `micrograd_plus` package.
> After understanding how each layer works, you can import them directly:
>
> ```python
> from micrograd_plus import Linear, ReLU, Sigmoid, Dropout, Sequential
> ```

In [ ]:
# Setup
import numpy as np
import sys
from pathlib import Path

# Robust path resolution - works regardless of working directory
def _find_module_root():
    """Find the module root directory containing micrograd_plus."""
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / 'micrograd_plus' / '__init__.py').exists():
            return str(parent)
    return str(Path.cwd().parent)

sys.path.insert(0, _find_module_root())

from micrograd_plus import Tensor
from micrograd_plus.utils import set_seed, gradient_check

# Set seed for reproducibility
set_seed(42)

---

## Part 1: The Module Base Class

All layers inherit from a common `Module` class that provides:
- Training/evaluation mode switching
- Parameter collection
- Gradient zeroing

In [ ]:
from typing import List

class Module:
    """
    Base class for all neural network layers.
    
    Key methods:
    - forward(x): Compute output (must be implemented by subclasses)
    - parameters(): Return list of trainable tensors
    - train()/eval(): Switch between training and evaluation modes
    """
    
    def __init__(self):
        self._training = True
    
    @property
    def training(self) -> bool:
        return self._training
    
    def train(self, mode: bool = True) -> 'Module':
        """Set training mode (affects Dropout, BatchNorm, etc.)"""
        self._training = mode
        return self
    
    def eval(self) -> 'Module':
        """Set evaluation mode"""
        return self.train(False)
    
    def forward(self, x: Tensor) -> Tensor:
        """Compute forward pass - must be implemented by subclasses"""
        raise NotImplementedError
    
    def __call__(self, x: Tensor) -> Tensor:
        """Make the layer callable: layer(x) calls layer.forward(x)"""
        return self.forward(x)
    
    def parameters(self) -> List[Tensor]:
        """Return list of trainable parameters"""
        return []
    
    def zero_grad(self) -> None:
        """Reset all parameter gradients to zero"""
        for p in self.parameters():
            p.zero_grad()

---

## Part 2: Linear Layer

The **Linear layer** (also called Dense or Fully Connected) computes:

$$y = xW + b$$

Where:
- $x$: Input of shape `(batch_size, in_features)`
- $W$: Weight matrix of shape `(in_features, out_features)`
- $b$: Bias vector of shape `(out_features,)`
- $y$: Output of shape `(batch_size, out_features)`

### Weight Initialization

Good initialization is crucial! We use **Xavier/Glorot initialization**:
$$W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{in} + n_{out}}}\right)$$

This keeps the variance of activations roughly constant across layers.

In [ ]:
class Linear(Module):
    """
    Fully connected (dense) layer.
    
    Computes: y = x @ W + b
    
    Args:
        in_features: Number of input features
        out_features: Number of output features
        bias: Whether to include bias term (default: True)
    
    Example:
        >>> layer = Linear(784, 256)
        >>> x = Tensor(np.random.randn(32, 784))  # Batch of 32
        >>> y = layer(x)  # Shape: (32, 256)
    """
    
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Xavier/Glorot initialization
        scale = np.sqrt(2.0 / (in_features + out_features))
        self.weight = Tensor(
            np.random.randn(in_features, out_features).astype(np.float32) * scale,
            requires_grad=True
        )
        
        if bias:
            self.bias = Tensor(
                np.zeros(out_features, dtype=np.float32),
                requires_grad=True
            )
        else:
            self.bias = None
    
    def forward(self, x: Tensor) -> Tensor:
        """Forward pass: y = x @ W + b"""
        out = x @ self.weight
        if self.bias is not None:
            out = out + self.bias
        return out
    
    def parameters(self) -> List[Tensor]:
        """Return weight and bias"""
        if self.bias is not None:
            return [self.weight, self.bias]
        return [self.weight]
    
    def __repr__(self):
        return f"Linear({self.in_features}, {self.out_features}, bias={self.bias is not None})"

In [ ]:
# Test Linear layer
set_seed(42)

# Create layer
linear = Linear(4, 3)
print(f"Layer: {linear}")
print(f"Weight shape: {linear.weight.shape}")
print(f"Bias shape: {linear.bias.shape}")

# Forward pass
x = Tensor(np.random.randn(2, 4).astype(np.float32), requires_grad=True)
y = linear(x)

print(f"\nInput shape: {x.shape}")
print(f"Output shape: {y.shape}")
print(f"Output:\n{y.data}")

# Backward pass
loss = y.sum()
loss.backward()

print(f"\nWeight gradient shape: {linear.weight.grad.shape}")
print(f"Bias gradient: {linear.bias.grad}")

### 🔍 What Just Happened?

1. **Forward**: We computed `y = x @ W + b` where:
   - `x` is (2, 4) - 2 samples, 4 features each
   - `W` is (4, 3) - transforms 4 features to 3
   - `b` is (3,) - one bias per output feature
   - `y` is (2, 3) - 2 samples, 3 outputs each

2. **Backward**: Since `loss = sum(y)`, each element of `y` contributes equally:
   - `dL/dW = x.T @ ones` - each weight gets gradient proportional to input
   - `dL/db = sum(ones)` - bias gradient is just the number of samples

---

## Part 3: Activation Functions

Activation functions introduce **non-linearity**. Without them, stacking linear layers would just be one big linear transformation!

### Why Non-linearity Matters

Linear functions can only model straight lines/planes. Real-world patterns are curved:
- The relationship between "hours studied" and "exam score" isn't linear (diminishing returns)
- Image features like "edges" require detecting patterns, not just weighted sums

In [ ]:
class ReLU(Module):
    """
    Rectified Linear Unit: ReLU(x) = max(0, x)
    
    The most popular activation function because:
    - Simple and fast
    - Gradient is 0 or 1 (no vanishing gradient for positive inputs)
    - Creates sparse representations
    """
    
    def forward(self, x: Tensor) -> Tensor:
        return x.relu()
    
    def __repr__(self):
        return "ReLU()"


class Sigmoid(Module):
    """
    Sigmoid: sigma(x) = 1 / (1 + e^(-x))
    
    Squashes input to range (0, 1). Good for:
    - Binary classification output
    - Gate mechanisms (like in LSTMs)
    """
    
    def forward(self, x: Tensor) -> Tensor:
        return x.sigmoid()
    
    def __repr__(self):
        return "Sigmoid()"


class Tanh(Module):
    """
    Hyperbolic tangent: tanh(x) = (e^x - e^(-x)) / (e^x + e^(-x))
    
    Squashes input to range (-1, 1). Zero-centered, unlike Sigmoid.
    """
    
    def forward(self, x: Tensor) -> Tensor:
        return x.tanh()
    
    def __repr__(self):
        return "Tanh()"

In [ ]:
# Visualize activation functions
import matplotlib.pyplot as plt

x_vals = np.linspace(-3, 3, 100)

# Compute activations
relu_vals = np.maximum(0, x_vals)
sigmoid_vals = 1 / (1 + np.exp(-x_vals))
tanh_vals = np.tanh(x_vals)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

axes[0].plot(x_vals, relu_vals, 'b-', linewidth=2)
axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0].axvline(x=0, color='k', linestyle='--', alpha=0.3)
axes[0].set_title('ReLU: max(0, x)')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].grid(True, alpha=0.3)

axes[1].plot(x_vals, sigmoid_vals, 'r-', linewidth=2)
axes[1].axhline(y=0.5, color='k', linestyle='--', alpha=0.3)
axes[1].axvline(x=0, color='k', linestyle='--', alpha=0.3)
axes[1].set_title('Sigmoid: 1/(1+e^(-x))')
axes[1].set_xlabel('x')
axes[1].set_ylim(-0.1, 1.1)
axes[1].grid(True, alpha=0.3)

axes[2].plot(x_vals, tanh_vals, 'g-', linewidth=2)
axes[2].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[2].axvline(x=0, color='k', linestyle='--', alpha=0.3)
axes[2].set_title('Tanh')
axes[2].set_xlabel('x')
axes[2].set_ylim(-1.1, 1.1)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Test activation gradients
relu = ReLU()
sigmoid = Sigmoid()

# ReLU gradient
x = Tensor([-2.0, -1.0, 0.0, 1.0, 2.0], requires_grad=True)
y = relu(x)
print(f"ReLU input:  {x.data}")
print(f"ReLU output: {y.data}")

y.sum().backward()
print(f"ReLU gradient: {x.grad}")
print("  (Gradient is 1 where input > 0, else 0)")

# Sigmoid gradient
x = Tensor([-2.0, 0.0, 2.0], requires_grad=True)
y = sigmoid(x)
print(f"\nSigmoid input:  {x.data}")
print(f"Sigmoid output: {y.data}")

y.sum().backward()
print(f"Sigmoid gradient: {x.grad}")
print(f"  (Gradient = sigmoid(x) * (1 - sigmoid(x)))")
print(f"  Expected: {y.data * (1 - y.data)}")

---

## Part 4: Softmax Layer

**Softmax** converts raw scores (logits) into a probability distribution:

$$\text{softmax}(x)_i = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

Properties:
- All outputs are positive
- All outputs sum to 1
- Higher input values get higher probabilities

In [ ]:
class Softmax(Module):
    """
    Softmax activation: converts logits to probabilities.
    
    softmax(x)_i = exp(x_i) / sum(exp(x_j))
    
    Args:
        axis: Dimension along which to compute softmax (default: -1)
    """
    
    def __init__(self, axis: int = -1):
        super().__init__()
        self.axis = axis
    
    def forward(self, x: Tensor) -> Tensor:
        return x.softmax(axis=self.axis)
    
    def __repr__(self):
        return f"Softmax(axis={self.axis})"


# Test Softmax
softmax = Softmax()

# Example: Classification scores for 3 classes
logits = Tensor([[2.0, 1.0, 0.1]], requires_grad=True)
probs = softmax(logits)

print(f"Logits (raw scores): {logits.data}")
print(f"Probabilities: {probs.data}")
print(f"Sum of probabilities: {probs.data.sum():.4f} (should be 1.0)")

# Batch example
batch_logits = Tensor([
    [2.0, 1.0, 0.1],  # Sample 1: clearly class 0
    [0.1, 0.2, 3.0],  # Sample 2: clearly class 2
    [1.0, 1.0, 1.0]   # Sample 3: uncertain (equal)
])
batch_probs = softmax(batch_logits)

print(f"\nBatch probabilities:")
for i, (logit, prob) in enumerate(zip(batch_logits.data, batch_probs.data)):
    pred = np.argmax(prob)
    print(f"  Sample {i}: logits={logit} -> probs={prob.round(3)} -> class {pred}")

---

## Part 5: Dropout Layer

**Dropout** is a regularization technique that randomly "drops" (sets to zero) a fraction of neurons during training.

### Why Dropout Works

1. **Prevents co-adaptation**: Neurons can't rely on specific other neurons
2. **Ensemble effect**: Each training step uses a different "sub-network"
3. **Implicit regularization**: Forces the network to be more robust

### Important: Inverted Dropout

During training, we scale up remaining activations by `1/(1-p)` so that expected values match. This way, we don't need to scale during inference.

In [ ]:
class Dropout(Module):
    """
    Dropout regularization layer.
    
    During training: randomly sets elements to 0 with probability p,
    and scales remaining elements by 1/(1-p).
    
    During evaluation: does nothing (identity function).
    
    Args:
        p: Probability of dropping each element (default: 0.5)
    """
    
    def __init__(self, p: float = 0.5):
        super().__init__()
        if not 0 <= p < 1:
            raise ValueError(f"p must be in [0, 1), got {p}")
        self.p = p
    
    def forward(self, x: Tensor) -> Tensor:
        # During eval, return unchanged
        if not self.training or self.p == 0:
            return x
        
        # Create random mask
        mask = (np.random.rand(*x.shape) > self.p).astype(np.float32)
        scale = 1.0 / (1.0 - self.p)
        
        # Apply mask with scaling
        out = Tensor(
            x.data * mask * scale,
            requires_grad=x.requires_grad,
            _children=(x,),
            _op='dropout'
        )
        
        def _backward():
            if x.requires_grad:
                # Gradient only flows through non-dropped elements
                x.grad += mask * scale * out.grad
        out._backward = _backward
        
        return out
    
    def __repr__(self):
        return f"Dropout(p={self.p})"

In [ ]:
# Test Dropout
set_seed(42)

dropout = Dropout(p=0.5)

x = Tensor(np.ones((2, 5)), requires_grad=True)
print(f"Input (all ones):\n{x.data}")

# Training mode
dropout.train()
y_train = dropout(x)
print(f"\nTraining mode output (50% dropped, rest scaled by 2):")
print(f"{y_train.data}")
print(f"Mean (should be ~1.0): {y_train.data.mean():.2f}")

# Evaluation mode
dropout.eval()
y_eval = dropout(x)
print(f"\nEval mode output (unchanged):")
print(f"{y_eval.data}")

# Show that training outputs vary
dropout.train()
print(f"\nMultiple training forward passes:")
for i in range(3):
    y = dropout(x)
    print(f"  Pass {i+1}: mean={y.data.mean():.2f}, zeros={np.sum(y.data == 0)}")

---

## Part 6: Sequential Container

The **Sequential** container lets us stack layers and treat them as a single model.

In [ ]:
class Sequential(Module):
    """
    Container that runs layers in sequence.
    
    Example:
        model = Sequential(
            Linear(784, 256),
            ReLU(),
            Dropout(0.2),
            Linear(256, 10)
        )
        output = model(input)
    """
    
    def __init__(self, *layers):
        super().__init__()
        self.layers = list(layers)
    
    def forward(self, x: Tensor) -> Tensor:
        """Pass input through all layers sequentially"""
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self) -> List[Tensor]:
        """Collect parameters from all layers"""
        params = []
        for layer in self.layers:
            params.extend(layer.parameters())
        return params
    
    def train(self, mode: bool = True):
        """Set training mode for all layers"""
        self._training = mode
        for layer in self.layers:
            layer.train(mode)
        return self
    
    def __repr__(self):
        lines = ["Sequential("]
        for i, layer in enumerate(self.layers):
            lines.append(f"  ({i}): {layer}")
        lines.append(")")
        return "\n".join(lines)
    
    def __getitem__(self, idx):
        return self.layers[idx]
    
    def __len__(self):
        return len(self.layers)

In [ ]:
# Build a simple MLP (Multi-Layer Perceptron)
set_seed(42)

model = Sequential(
    Linear(4, 8),     # Input layer: 4 features -> 8 hidden
    ReLU(),
    Dropout(0.2),
    Linear(8, 8),     # Hidden layer
    ReLU(),
    Linear(8, 3)      # Output layer: 3 classes
)

print("Model architecture:")
print(model)

# Count parameters
params = model.parameters()
total_params = sum(p.size for p in params)
print(f"\nTotal parameters: {total_params}")
for i, p in enumerate(params):
    print(f"  Param {i}: shape={p.shape}, size={p.size}")

In [ ]:
# Forward pass through the model
x = Tensor(np.random.randn(2, 4).astype(np.float32), requires_grad=True)

# Training mode
model.train()
y_train = model(x)
print(f"Training mode output:")
print(f"  Input shape: {x.shape}")
print(f"  Output shape: {y_train.shape}")
print(f"  Output:\n{y_train.data}")

# Backward pass
loss = y_train.sum()
loss.backward()

print(f"\nGradients computed:")
for i, p in enumerate(model.parameters()):
    grad_norm = np.sqrt(np.sum(p.grad ** 2))
    print(f"  Param {i}: grad_norm={grad_norm:.4f}")

# Eval mode
model.eval()
y_eval = model(x)
print(f"\nEval mode output (deterministic):")
print(f"{y_eval.data}")

---

## Try It Yourself: Exercise

Implement a **LeakyReLU** activation function:

$$\text{LeakyReLU}(x) = \begin{cases} x & \text{if } x > 0 \\ \alpha x & \text{if } x \leq 0 \end{cases}$$

Where $\alpha$ is a small positive number (typically 0.01).

<details>
<summary>Hint</summary>

The gradient is:
- 1 where input > 0
- alpha where input <= 0

You can implement this using `np.where` or by combining masks.
</details>

In [ ]:
# YOUR CODE HERE
class LeakyReLU(Module):
    """
    Leaky ReLU: allows small gradient for negative inputs.
    
    Args:
        alpha: Slope for negative inputs (default: 0.01)
    """
    
    def __init__(self, alpha: float = 0.01):
        super().__init__()
        self.alpha = alpha
    
    def forward(self, x: Tensor) -> Tensor:
        # TODO: Implement LeakyReLU
        # Hint: Use np.where or combine x.relu() with x * alpha
        pass

# Test your implementation
# leaky_relu = LeakyReLU(alpha=0.1)
# x = Tensor([-2, -1, 0, 1, 2], requires_grad=True)
# y = leaky_relu(x)
# print(f"Input: {x.data}")
# print(f"Output: {y.data}")  # Should be [-0.2, -0.1, 0, 1, 2]

---

## Common Mistakes

### Mistake 1: Forgetting to Switch Modes

```python
# Wrong: Using training mode for inference
model = Sequential(Linear(10, 5), Dropout(0.5), Linear(5, 2))
# Dropout will randomly drop neurons!
prediction = model(test_input)

# Right: Switch to eval mode
model.eval()
prediction = model(test_input)
```

### Mistake 2: Wrong Weight Initialization

```python
# Wrong: Too large initial weights
weight = Tensor(np.random.randn(100, 50) * 10, requires_grad=True)  # Too big!

# Wrong: Too small initial weights
weight = Tensor(np.random.randn(100, 50) * 0.0001, requires_grad=True)  # Too small!

# Right: Xavier/Glorot initialization
scale = np.sqrt(2.0 / (100 + 50))
weight = Tensor(np.random.randn(100, 50) * scale, requires_grad=True)
```

### Mistake 3: Applying Softmax Twice

```python
# Wrong: CrossEntropyLoss already includes softmax!
logits = model(x)
probs = softmax(logits)  # Don't do this!
loss = cross_entropy(probs, targets)  # Double softmax = wrong gradients

# Right: Pass raw logits to CrossEntropyLoss
logits = model(x)
loss = cross_entropy(logits, targets)
```

---

## Checkpoint

You've learned:
- How to structure neural network layers with the Module base class
- How Linear layers transform features with learnable weights
- Why activation functions (ReLU, Sigmoid, Softmax) are essential
- How Dropout regularization works and why it helps
- How to combine layers into a Sequential model

---

## Challenge (Optional)

Implement **Batch Normalization**:

$$\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$
$$y = \gamma \hat{x} + \beta$$

Where:
- $\mu_B, \sigma_B^2$ are batch mean and variance
- $\gamma, \beta$ are learnable scale and shift parameters
- During eval, use running averages instead of batch statistics

---

## Further Reading

- [Understanding Xavier Initialization](https://cs231n.github.io/neural-networks-2/#init)
- [Dropout Paper (Srivastava et al., 2014)](https://jmlr.org/papers/v15/srivastava14a.html)
- [Batch Normalization Paper (Ioffe & Szegedy, 2015)](https://arxiv.org/abs/1502.03167)

---

## Cleanup

In [ ]:
# Cleanup - release memory
from micrograd_plus.utils import cleanup_notebook
cleanup_notebook(globals())

---

## Next Steps

Now that you understand neural network layers, we'll build on this in:
- **Lab 1.7.3**: Loss functions and optimizers
- **Lab 1.7.4**: Testing suite for verifying your implementation